In [1]:
import sys
import wandb
from omegaconf import OmegaConf as omg
import torch
import numpy as np
from agent import Agent
from trainer import Trainer, reward_fn
from data_generator_v2 import DataGenerator, SimulatedAnnealing
import time
import os
import pandas as pd

In [2]:

def load_conf():
    """Quick method to load configuration (using OmegaConf). By default,
    configuration is loaded from the default config file (config.yaml).
    Another config file can be specific through command line.
    Also, configuration can be over-written by command line.

    Returns:
        OmegaConf.DictConfig: OmegaConf object representing the configuration.
    """
    default_conf = omg.create({"config" : "config.yaml"})

    sys.argv = [a.strip("-") for a in sys.argv]
    cli_conf = omg.from_cli()

    yaml_file = omg.merge(default_conf, cli_conf).config

    yaml_conf = omg.load(yaml_file)

    return omg.merge(default_conf, yaml_conf, cli_conf)

In [3]:
def main():
    conf = load_conf()
    wandb.init(project=conf.proj_name, config=dict(conf))

    agent = Agent(space_dim= conf.dimension, embed_hidden=conf.embed_hidden, enc_stacks=conf.enc_stacks, ff_hidden=conf.ff_hidden, enc_heads=conf.enc_heads, query_hidden=conf.query_hidden, att_hidden=conf.att_hidden, crit_hidden=conf.crit_hidden, n_history=conf.n_history, p_dropout=conf.p_dropout)
    wandb.watch(agent)

    dataset = DataGenerator()

    trainer = Trainer(conf, agent, dataset)
    trainer.run()

    # Save trained agent
    dir_ = str(conf.dimension)+'D_'+'MESH'+str(conf.max_len) +'_b'+str(conf.batch_size)+'_e'+str(conf.embed_hidden)+'_n'+str(conf.ff_hidden)+'_s'+str(conf.enc_stacks)+'_h'+str(conf.enc_heads)+ '_q'+str(conf.query_hidden) +'_a'+str(conf.att_hidden)+'_c'+str(conf.crit_hidden)+ '_lr'+str(conf.lr)+'_d'+str(conf.lr_decay_steps)+'_'+str(conf.lr_decay_rate)+ '_steps'+str(conf.steps)
    path = "save/"+dir_
    if not os.path.exists(path):
        os.makedirs(path)
    save_path= str(path)+'/'+str(conf.model_path)
    torch.save(agent.state_dict(), save_path)

    input_test = []
    for _ in range (conf.batch_size): 
        input_test1 = np.loadtxt("mpeg_4x3.txt", delimiter=",")
        input_test.append(input_test1)

    if conf.test:
        device = torch.device(conf.device)
        # Load trained agent
        agent.load_state_dict(torch.load(save_path))
        agent.eval()
        agent = agent.to(device)
        start_time = time.time()
        running_reward = 0
        for _ in range(conf.test_steps):
            #input_batch = dataset.test_batch(conf.batch_size, conf.max_len, conf.dimension, shuffle=False)
            input_batch = torch.Tensor(input_test).to(device)

            tour, *_ = agent(input_batch)

            reward = reward_fn(input_batch, tour, conf.x_dim, conf.batch_size)

            # Find best solution
            j = reward.argmin()
            best_tour = tour[j][:].tolist()

            # Log
            running_reward += reward[j]

            # Display
            print('Reward (before 2 opt)', reward[j],'tour', best_tour)
            opt_tour, opt_length = dataset.loop2opt(input_batch.cpu()[j], best_tour, conf.x_dim)
            print('Reward (with 2 opt)', opt_length, 'opt_tour',opt_tour)
            #dataset.visualize_2D_trip(opt_tour)

        wandb.run.summary["test_reward"] = running_reward / conf.test_steps
        print("--- %s seconds ---" % (time.time() - start_time))

In [4]:
if __name__ == "__main__":
    main()

wandb: Currently logged in as: jagadheesh27 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.29 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


C:\Anaconda3\envs\tf\lib\site-packages\torch\nn\modules\module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


Reward (before 2 opt) tensor(5893.) tour [6, 9, 4, 7, 2, 10, 8, 3, 11, 5, 1, 0]
Reward (with 2 opt) [4456.] opt_tour [ 6  7  9  5 11  3  4  2 10  8  0  1]
Reward (before 2 opt) tensor(5624.) tour [6, 11, 3, 0, 9, 7, 4, 2, 1, 8, 5, 10]
Reward (with 2 opt) [4645.] opt_tour [ 6 11  3  0  7  9  4  8 10  5  2  1]
Reward (before 2 opt) tensor(6137.) tour [6, 0, 5, 10, 8, 9, 4, 3, 11, 1, 7, 2]
Reward (with 2 opt) [5251.] opt_tour [ 6 10  0  5  8  9  4  3 11  7  2  1]
Reward (before 2 opt) tensor(6233.) tour [6, 9, 4, 11, 10, 3, 8, 0, 5, 2, 7, 1]
Reward (with 2 opt) [4214.] opt_tour [ 6  9  4  8 11 10  3  2  7  1  0  5]
Reward (before 2 opt) tensor(5110.) tour [6, 11, 7, 1, 9, 3, 10, 8, 0, 4, 5, 2]
Reward (with 2 opt) [4025.] opt_tour [ 6 11  7  1  9  4  0  8 10  3  2  5]
Reward (before 2 opt) tensor(5560.) tour [6, 9, 4, 0, 7, 2, 3, 11, 5, 10, 1, 8]
Reward (with 2 opt) [4413.] opt_tour [ 6 11 10  5  7  9  4  3  8  1  0  2]
Reward (before 2 opt) tensor(5452.) tour [6, 9, 2, 8, 3, 4, 0, 10, 7, 

In [39]:
import torch.nn as nn
dataset = DataGenerator()
input_test = dataset.train_batch(10, 12, 12, 4, 3)
input_batch  = torch.Tensor(input_test).to('cpu')
m = nn.Conv1d(12, 128, 1, stride=1)
n = nn.Linear(12, 128)
input = torch.randn(20, 16, 50)
output = m(input_batch)#.transpose(1, 2)
output1 = n(input_batch).transpose(1, 2)

In [41]:
np.shape(output1)

torch.Size([10, 128, 12])

In [14]:
    
    temp = 1000
    stopping_temp = 0.00001
    alpha = 0.995
    stopping_iter = 10000
    conf = load_conf()
    dataset = DataGenerator()
    agent = Agent(space_dim= conf.dimension, embed_hidden=conf.embed_hidden, enc_stacks=conf.enc_stacks, ff_hidden=conf.ff_hidden, enc_heads=conf.enc_heads, query_hidden=conf.query_hidden, att_hidden=conf.att_hidden, crit_hidden=conf.crit_hidden, n_history=conf.n_history, p_dropout=conf.p_dropout)
    dir_ = str(conf.dimension)+'D_'+'MESH'+str(conf.max_len) +'_b'+str(conf.batch_size)+'_e'+str(conf.embed_hidden)+'_n'+str(conf.ff_hidden)+'_s'+str(conf.enc_stacks)+'_h'+str(conf.enc_heads)+ '_q'+str(conf.query_hidden) +'_a'+str(conf.att_hidden)+'_c'+str(conf.crit_hidden)+ '_lr'+str(conf.lr)+'_d'+str(conf.lr_decay_steps)+'_'+str(conf.lr_decay_rate)+ '_steps'+str(conf.steps)
    path = "save/"+dir_
    save_path= str(path)+'/'+str(conf.model_path)
    #input_test = dataset.train_batch(conf.batch_size-1, conf.max_len, conf.dimension, conf.x_dim, conf.y_dim)
    input_test = []
    for _ in range (128):#conf.batch_size):        
        A=np.zeros((conf.max_len,conf.max_len),dtype='float')
        app = pd.read_csv('mpeg.csv')
        for i in range ( len(app)):
            A[app.source[i]][app.target[i]]= app.weight[i]
        input_test.append(A)

    if conf.test:
        device = torch.device(conf.device)
        # Load trained agent
        agent.load_state_dict(torch.load(save_path))
        agent.eval()
        agent = agent.to(device)
        start_time = time.time()
        running_reward = 0
        for _ in range(1):#conf.test_steps):
            #input_batch = dataset.test_batch(conf.batch_size, conf.max_len, conf.dimension, shuffle=False)
            input_batch = torch.Tensor(input_test).to(device)

            tour, *_ = agent(input_batch)

            reward = reward_fn(input_batch, tour, conf.x_dim, conf.batch_size)

            # Find best solution
            j = reward.argmin()
            #j=127
            best_tour = tour[j][:].tolist()

            # Log
            running_reward += reward[j]

            # Display
            print('Reward (before 2 opt)', reward[j],'tour', best_tour)
            print("--- %s seconds ---" % (time.time() - start_time))
            opt_tour, opt_length = dataset.loop2opt(input_batch.cpu()[j], best_tour, conf.x_dim)
            print('Reward (with 2 opt)', opt_length, 'opt_tour',opt_tour)
            print("--- %s seconds ---" % (time.time() - start_time))
            #sa = SimulatedAnnealing(input_batch.cpu()[j], best_tour, conf.x_dim, temp, alpha, stopping_temp, stopping_iter)
            #SA_length, SA_tour = sa.anneal()
            #print('Reward (with SA)', SA_length, 'SA_tour',SA_tour)
            #print("--- %s seconds ---" % (time.time() - start_time))
            #dataset.visualize_2D_trip(opt_tour)
        
        print("---total time %s seconds ---" % (time.time() - start_time))

Reward (before 2 opt) tensor(5253.5000) tour [1, 4, 0, 9, 8, 2, 11, 6, 5, 3, 10, 7]
--- 0.6699981689453125 seconds ---
Reward (with 2 opt) [4617.5] opt_tour [ 1  0  4  9  8  2 11  6  5 10  3  7]
--- 1.134000301361084 seconds ---
---total time 1.1349973678588867 seconds ---


In [10]:
reward

tensor([31411.0000, 15861.0000, 25629.0000, 21178.0000, 22216.0000, 25314.0000,
        35372.0000, 16938.0000, 19206.0000, 24354.0000, 17212.0000, 20431.0000,
        18271.0000, 24972.0000, 19832.0000, 26592.0000, 21302.0000, 15859.0000,
        19294.0000, 19172.0000, 15141.0000, 22694.0000, 17635.0000, 22763.0000,
        25077.0000, 23137.0000, 22828.0000, 28864.0000, 12551.0000, 23071.0000,
        20330.0000, 19898.0000, 22069.0000, 28870.0000, 27339.0000, 13757.0000,
        18659.0000, 24120.0000, 16651.0000, 23384.0000, 21281.0000, 27780.0000,
        30325.0000, 26272.0000, 24135.0000, 23200.0000, 24802.0000, 29562.0000,
        27182.0000, 25323.0000, 19153.0000, 24146.0000, 21987.0000, 30765.0000,
        33482.0000, 27550.0000, 23741.0000, 24274.0000, 20677.0000, 25385.0000,
        25685.0000, 21290.0000, 22664.0000, 29846.0000, 18533.0000, 16804.0000,
        25279.0000, 19076.0000, 19567.0000, 16952.0000, 20323.0000, 18554.0000,
        18659.0000, 22474.0000, 20081.00

In [9]:
conf = load_conf()
dir_ = str(conf.dimension)+'D_'+'MESH'+str(conf.max_len) +'_b'+str(conf.batch_size)+'_e'+str(conf.embed_hidden)+'_n'+str(conf.ff_hidden)+'_s'+str(conf.enc_stacks)+'_h'+str(conf.enc_heads)+ '_q'+str(conf.query_hidden) +'_a'+str(conf.att_hidden)+'_c'+str(conf.crit_hidden)+ '_lr'+str(conf.lr)+'_d'+str(conf.lr_decay_steps)+'_'+str(conf.lr_decay_rate)+ '_steps'+str(conf.steps)
print(dir_)
path = "save/"+dir_
if not os.path.exists(path):
    os.makedirs(path)
#save_path= str(path)+'/'+str(conf.model_path)
#print (path)
from shutil import copy
copy('config.yaml',path)

12D_MESH12_b128_e128_n512_s3_h16_q360_a256_c256_lr0.001_d5000_0.96_steps30000


'save/12D_MESH12_b128_e128_n512_s3_h16_q360_a256_c256_lr0.001_d5000_0.96_steps30000\\config.yaml'

In [19]:
import pandas as pd
A=np.zeros((12,12),dtype='float')
app = pd.read_csv('mwd.csv')
for i in range ( len(app)):
    A[app.source[i]][app.target[i]]= app.weight[i]
    A[app.target[i]][app.source[i]]= app.weight[i]

AttributeError: 'DataFrame' object has no attribute 'weight'

In [27]:
        
        import random
        import networkx as nx
        input_batch = []
        

        size1 = 12 #mesh size 4x3
        for _ in range(1):        
            size = random.randint((size1 - 4),size1)

            G=nx.connected_watts_strogatz_graph(size, 5, 0.5, tries=100, seed=None)
            for (u, v) in G.edges():
                G.edges[u,v]['weight'] = random.randint(1,1000)
            #C=G.to_undirected()
            #B=nx.to_numpy_array(C)
            app= nx.to_pandas_edgelist(G)
            A=np.zeros((size1,size1),dtype='float')
            for i in range (len(app)):
                A[app.source[i]][app.target[i]]= app.weight[i]
            #A[:B.shape[0], :B.shape[1]] = B
            #A=A.flatten()
            input_batch.append(A)
            gg=nx.from_numpy_array(A)
            app1 = nx.to_pandas_edgelist(gg)

In [11]:
dataset = DataGenerator()
input_test = dataset.train_batch(10, 12, 12, 4, 3)


In [12]:
print(input_test[9])

[[0.00e+00 0.00e+00 0.00e+00 0.00e+00 1.90e+02 0.00e+00 0.00e+00 0.00e+00
  0.00e+00 0.00e+00 0.00e+00 0.00e+00]
 [0.00e+00 0.00e+00 0.00e+00 0.00e+00 5.00e-01 0.00e+00 0.00e+00 0.00e+00
  0.00e+00 0.00e+00 0.00e+00 0.00e+00]
 [0.00e+00 0.00e+00 0.00e+00 0.00e+00 6.00e+01 4.00e+01 0.00e+00 0.00e+00
  0.00e+00 0.00e+00 0.00e+00 0.00e+00]
 [0.00e+00 0.00e+00 0.00e+00 0.00e+00 6.00e+02 4.00e+01 0.00e+00 0.00e+00
  0.00e+00 0.00e+00 0.00e+00 0.00e+00]
 [0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
  5.00e-01 9.10e+02 3.20e+01 0.00e+00]
 [0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
  0.00e+00 0.00e+00 0.00e+00 0.00e+00]
 [0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 2.50e+02
  0.00e+00 6.70e+02 1.73e+02 5.00e+02]
 [0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
  0.00e+00 0.00e+00 0.00e+00 0.00e+00]
 [0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00 0.00e+00
  0.00e+00 0.00e+00 0.